<img width="400" src="https://nomeroff.net.ua/images/logo/nomeroff_net.svg" alt="Nomeroff Net. Automatic numberplate recognition system" align="left"/>

## The example demonstrates license plate number detection.

Before runing this demo, please download models from [https://nomeroff.net.ua/models/](https://nomeroff.net.ua/models/) to **./models/** directory. 

In [9]:
# Import all necessary libraries.
import os
import cv2
import numpy as np
import sys
import json
import matplotlib.image as mpimg
from matplotlib import pyplot as plt
import warnings
import os
warnings.filterwarnings('ignore')

# change this property
NOMEROFF_NET_DIR = os.path.abspath('../../../')

# specify the path to Mask_RCNN if you placed it outside Nomeroff-net project
MASK_RCNN_DIR = os.path.join(NOMEROFF_NET_DIR, 'Mask_RCNN')

MASK_RCNN_LOG_DIR = os.path.join(NOMEROFF_NET_DIR, 'logs')
MASK_RCNN_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, "models/mask_rcnn_numberplate_0700.h5")
OPTIONS_MODEL_PATH =  os.path.join(NOMEROFF_NET_DIR, "models/numberplate_options_2019_05_20.h5")

# If you use gpu version tensorflow please change model to gpu version named like *-gpu.pb
mode = "cpu"
OCR_NP_UKR_TEXT = os.path.join(NOMEROFF_NET_DIR, "models/anpr_ocr_ua_12-{}.h5".format(mode))
OCR_NP_EU_TEXT =  os.path.join(NOMEROFF_NET_DIR, "models/anpr_ocr_eu_2-{}.h5".format(mode))
OCR_NP_RU_TEXT =  os.path.join(NOMEROFF_NET_DIR, "models/anpr_ocr_ru_3-{}.h5".format(mode))
OCR_NP_KZ_TEXT =  os.path.join(NOMEROFF_NET_DIR, "models/anpr_ocr_kz_4-{}.h5".format(mode))

sys.path.append(NOMEROFF_NET_DIR)

Also you may use .pb models for options detectors or OCR detectors. 
More about conversion and preparing for inference withit https://github.com/ria-com/nomeroff-net/blob/master/examples/inference_convert.ipynb.

In [10]:
# Import license plate recognition tools.
from NomeroffNet import  filters, RectDetector, TextDetector, OptionsDetector, Detector, textPostprocessing, textPostprocessingAsync

# Initialize npdetector with default configuration file.
nnet = Detector(MASK_RCNN_DIR, MASK_RCNN_LOG_DIR)
nnet.loadModel(MASK_RCNN_MODEL_PATH)

rectDetector = RectDetector()

optionsDetector = OptionsDetector({
    "class_region": ["xx_unknown", "eu_ua_2015", "eu_ua_2004", "eu_ua_1995", "eu", "xx_transit", "ru", "kz"]
})
optionsDetector.load(OPTIONS_MODEL_PATH)

# Initialize text detector.
textDetector = TextDetector({
    "eu_ua_2004_2015": {
        "for_regions": ["eu_ua_2015", "eu_ua_2004"],
        "model_path": OCR_NP_UKR_TEXT
    },
    "eu": {
        "for_regions": ["eu", "eu_ua_1995"],
        "model_path": OCR_NP_EU_TEXT
    },
    "ru": {
        "for_regions": ["ru"],
        "model_path": OCR_NP_RU_TEXT
    },
    "kz": {
        "for_regions": ["kz"],
        "model_path": OCR_NP_KZ_TEXT
    }
})

W0522 16:47:31.232988 140046717769024 deprecation.py:237] From /usr/local/lib64/python3.6/site-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.



In [ ]:
# Walking through the ./examples/images/ directory and checking each of the images for license plates.
rootDir = '/var/www/datasets/fake-region/dnr/'

max_img_w = 1600
for dirName, subdirList, fileList in os.walk(rootDir):
    for fname in fileList:
        baseName = os.path.splitext(os.path.basename(fname))[0]
        img_path = os.path.join(dirName, fname)
        print(img_path)
        img = mpimg.imread(img_path)
        #plt.axis("off")
        #plt.imshow(img)
        #plt.show()
         
        # corect size for better speed
        img_w = img.shape[1]
        img_h = img.shape[0]
        img_w_r = 1
        img_h_r = 1
        if img_w > max_img_w:
            resized_img = cv2.resize(img, (max_img_w, int(max_img_w/img_w*img_h)))
            img_w_r = img_w/max_img_w
            img_h_r = img_h/(max_img_w/img_w*img_h)
        else:
            resized_img = img
        
        try:
            NP = nnet.detect([resized_img]) 

            # Generate image mask.
            cv_img_masks = await filters.cv_img_mask_async(NP)

            # Detect points.
            arrPoints = await rectDetector.detectAsync(cv_img_masks, outboundHeightOffset=3-img_w_r, fixGeometry=True, fixRectangleAngle=10)
            #print(arrPoints)
            arrPoints[..., 1:2] = arrPoints[..., 1:2]*img_h_r
            arrPoints[..., 0:1] = arrPoints[..., 0:1]*img_w_r

            # cut zones
            zones = await rectDetector.get_cv_zonesBGR_async(img, arrPoints)
            toShowZones = await rectDetector.get_cv_zonesRGB_async(img, arrPoints)

            # find standart
            regionIds, stateIds, countLines = optionsDetector.predict(zones)
            regionNames = optionsDetector.getRegionLabels(regionIds)
            foundNumber=0
            for zone, points, regionId in zip(toShowZones, arrPoints, regionIds):
                #plt.axis("off")
                    mpimg.imsave("/var/www/datasets/fake-region/dnr/img/{}_{}.png".format(baseName, foundNumber), zone)
                    foundNumber += 1
                    #plt.imshow(zone)
                #plt.show()


            #print(regionNames)

            # find text with postprocessing by standart  
            textArr = textDetector.predict(zones, regionNames, countLines)
            textArr = await textPostprocessingAsync(textArr, regionNames)
            #print(textArr)
        except Exception as e:
            print("Error")
            print(e)

/var/www/datasets/fake-region/dnr/12010-main.jpg
/var/www/datasets/fake-region/dnr/10214-main.jpg
/var/www/datasets/fake-region/dnr/12985-main.jpg
/var/www/datasets/fake-region/dnr/13094-main.jpg
Error
division by zero
/var/www/datasets/fake-region/dnr/9780-main.jpg
/var/www/datasets/fake-region/dnr/10119-main.jpg
Error
division by zero
/var/www/datasets/fake-region/dnr/9884-main.jpg
/var/www/datasets/fake-region/dnr/9800-main.jpg
/var/www/datasets/fake-region/dnr/13325-main.jpg
/var/www/datasets/fake-region/dnr/11429-main.jpg
/var/www/datasets/fake-region/dnr/12538-main.jpg
/var/www/datasets/fake-region/dnr/12399-main.jpg
Error
division by zero
/var/www/datasets/fake-region/dnr/10172-main.jpg
/var/www/datasets/fake-region/dnr/12859-main.jpg
/var/www/datasets/fake-region/dnr/12454-main.jpg
/var/www/datasets/fake-region/dnr/9995-main.jpg
/var/www/datasets/fake-region/dnr/12122-main.jpg
/var/www/datasets/fake-region/dnr/12488-main.jpg
/var/www/datasets/fake-region/dnr/11868-main.jpg
Erro

/var/www/datasets/fake-region/dnr/13249-main.jpg
/var/www/datasets/fake-region/dnr/13464-main.jpg
/var/www/datasets/fake-region/dnr/11902-main.jpg
Error
division by zero
/var/www/datasets/fake-region/dnr/12977-main.jpg
/var/www/datasets/fake-region/dnr/10182-main.jpg
/var/www/datasets/fake-region/dnr/12244-main.jpg
Error
division by zero
/var/www/datasets/fake-region/dnr/9874-main.jpg
/var/www/datasets/fake-region/dnr/12196-main.jpg
/var/www/datasets/fake-region/dnr/9963-main.jpg
/var/www/datasets/fake-region/dnr/11816-main.jpg
Error
division by zero
/var/www/datasets/fake-region/dnr/11605-main.jpg
Error
division by zero
/var/www/datasets/fake-region/dnr/12368-main.jpg
Error
division by zero
/var/www/datasets/fake-region/dnr/12078-main.jpg
Error
division by zero
/var/www/datasets/fake-region/dnr/13360-main.jpg
/var/www/datasets/fake-region/dnr/13428-main.jpg
/var/www/datasets/fake-region/dnr/12447-main.jpg
/var/www/datasets/fake-region/dnr/12682-main.jpg
/var/www/datasets/fake-region/d

Error
division by zero
/var/www/datasets/fake-region/dnr/12588-main.jpg
Error
division by zero
/var/www/datasets/fake-region/dnr/11357-main.jpg
/var/www/datasets/fake-region/dnr/12997-main.jpg
/var/www/datasets/fake-region/dnr/13016-main.jpg
/var/www/datasets/fake-region/dnr/12798-main.jpg
Error
division by zero
/var/www/datasets/fake-region/dnr/12862-main.jpg
/var/www/datasets/fake-region/dnr/12612-main.jpg
/var/www/datasets/fake-region/dnr/12831-main.jpg
Error
division by zero
/var/www/datasets/fake-region/dnr/13242-main.jpg
Error
division by zero
/var/www/datasets/fake-region/dnr/11715-main.jpg
/var/www/datasets/fake-region/dnr/12801-main.jpg
/var/www/datasets/fake-region/dnr/12232-main.jpg
/var/www/datasets/fake-region/dnr/10375-main.jpg
/var/www/datasets/fake-region/dnr/13157-main.jpg
/var/www/datasets/fake-region/dnr/11875-main.jpg
/var/www/datasets/fake-region/dnr/10017-main.jpg
/var/www/datasets/fake-region/dnr/12645-main.jpg
/var/www/datasets/fake-region/dnr/12119-main.jpg
/va

/var/www/datasets/fake-region/dnr/12676-main.jpg
/var/www/datasets/fake-region/dnr/12749-main.jpg
/var/www/datasets/fake-region/dnr/10071-main.jpg
/var/www/datasets/fake-region/dnr/12584-main.jpg
/var/www/datasets/fake-region/dnr/12445-main.jpg
Error
division by zero
/var/www/datasets/fake-region/dnr/10224-main.jpg
Error
division by zero
/var/www/datasets/fake-region/dnr/12810-main.jpg
/var/www/datasets/fake-region/dnr/12759-main.jpg
/var/www/datasets/fake-region/dnr/13057-main.jpg
/var/www/datasets/fake-region/dnr/11892-main.jpg
/var/www/datasets/fake-region/dnr/12654-main.jpg
/var/www/datasets/fake-region/dnr/11984-main.jpg
/var/www/datasets/fake-region/dnr/13339-main.jpg
Error
division by zero
/var/www/datasets/fake-region/dnr/10346-main.jpg
Error
division by zero
/var/www/datasets/fake-region/dnr/10515-main.jpg
/var/www/datasets/fake-region/dnr/10210-main.jpg
/var/www/datasets/fake-region/dnr/13007-main.jpg
Error
division by zero
/var/www/datasets/fake-region/dnr/12156-main.jpg
/va

/var/www/datasets/fake-region/dnr/9841-main.jpg
/var/www/datasets/fake-region/dnr/13036-main.jpg
/var/www/datasets/fake-region/dnr/11791-main.jpg
/var/www/datasets/fake-region/dnr/11750-main.jpg
/var/www/datasets/fake-region/dnr/11585-main.jpg
Error
division by zero
/var/www/datasets/fake-region/dnr/9786-main.jpg
/var/www/datasets/fake-region/dnr/12610-main.jpg
/var/www/datasets/fake-region/dnr/10281-main.jpg
Error
division by zero
/var/www/datasets/fake-region/dnr/10275-main.jpg
/var/www/datasets/fake-region/dnr/13199-main.jpg
/var/www/datasets/fake-region/dnr/10236-main.jpg
/var/www/datasets/fake-region/dnr/12261-main.jpg
/var/www/datasets/fake-region/dnr/11501-main.jpg
/var/www/datasets/fake-region/dnr/12657-main.jpg
/var/www/datasets/fake-region/dnr/12032-main.jpg
Error
division by zero
/var/www/datasets/fake-region/dnr/13376-main.jpg
/var/www/datasets/fake-region/dnr/11517-main.jpg
/var/www/datasets/fake-region/dnr/12285-main.jpg
/var/www/datasets/fake-region/dnr/9829-main.jpg
/va